In [0]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.preprocessing import OneHotEncoder


In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
!cp "/gdrive/My Drive/Colab Notebooks/data/assistments12/q_mat.npz" .
!cp "/gdrive/My Drive/Colab Notebooks/data/assistments12/preprocessed_data.csv" .

In [0]:
df = pd.read_csv('preprocessed_data.csv', sep="\t")
qmat = sparse.load_npz('q_mat.npz').toarray()


In [0]:
# Transform q-matrix into dictionary
dict_q_mat = {i:set() for i in range(qmat.shape[0])}
for elt in np.argwhere(qmat == 1):
  dict_q_mat[elt[0]].add(elt[1])


In [0]:
X={}
X['df'] = np.empty((0,5)) # Keep track of the original dataset
for stud_id in df["user_id"].unique():
  df_stud = df[df["user_id"]==stud_id][["user_id", "item_id", "timestamp", "correct", "inter_id"]].copy()
  df_stud.sort_values(by="timestamp", inplace=True) # Sort values 
  df_stud = np.array(df_stud)
  X['df'] = np.vstack((X['df'], df_stud))


In [12]:
onehot = OneHotEncoder()
X['users'] = onehot.fit_transform(X["df"][:,0].reshape(-1,1))
print("Users encoded.")
X['items'] = onehot.fit_transform(X["df"][:,1].reshape(-1,1))
print("Items encoded.")
sparse_df = sparse.hstack([sparse.csr_matrix(X['df']),sparse.hstack([X['users'], X['items']])]).tocsr()


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Users encoded.


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Items encoded.


In [0]:
sparse.save_npz('X-IRT.npz', sparse_df)

In [0]:
!cp "X-IRT.npz" "/gdrive/My Drive/Colab Notebooks/data/assistments12/"